<a href="https://colab.research.google.com/github/eye-fly/BF-Presets/blob/main/docs/DNN-Lab-3-mnist-again-student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

In this lab, you will implement some of the techniques discussed in the lecture.

Below you are given a solution to the previous scenario. Note that it has two serious drawbacks:
 * The output predictions do not sum up to one (i.e. it does not return a distribution) even though the images always contain exactly one digit.
 * It uses MSE coupled with output sigmoid which can lead to saturation and slow convergence

**Task 0.** Implement a numerically stable version of softmax.

**Task 1.** Use softmax instead of coordinate-wise sigmoid and use log-loss instead of MSE. Test to see if this improves convergence. Hint: When implementing backprop it might be easier to consider these two functions as a single block and not even compute the gradient over the softmax values.

**Task 2.** Implement L2 regularization and add momentum to the SGD algorithm. Play with different amounts of regularization and momentum. See if this improves accuracy/convergence.

**Task 3 (optional).** Implement Adagrad or AdamW (currently popular in LLM training), dropout, and some simple data augmentations (e.g. tiny rotations/shifts, etc.). Again, test to see how these changes improve accuracy/convergence.

**Task 4.** Try adding extra layers to the network. Again, test how the changes you introduced affect accuracy/convergence. As a start, you can try this architecture: [784,100,30,10]

The provided model evaluation code (`evaluate_model`) may take some time to complete. During implementation, you can change the number of evaluated models to 1 and reduce the number of tested learning rates, and epochs.  


In [ ]:
!pip install tqdm
!pip install pandas

In [ ]:
import random
import numpy as np
from torchvision import datasets, transforms
from typing import List, Any, Tuple, Optional, Callable, Dict
from numpy.typing import NDArray
import json
from tqdm import tqdm
import pandas as pd

In [ ]:
# Let's read the mnist dataset


def load_mnist(path: str = "."):
    train_set = datasets.MNIST(path, train=True, download=True)
    x_train = train_set.data.numpy()
    _y_train = train_set.targets.numpy()

    test_set = datasets.MNIST(path, train=False, download=True)
    x_test = test_set.data.numpy()
    _y_test = test_set.targets.numpy()

    x_train = x_train.reshape((x_train.shape[0], 28 * 28)) / 255.0
    x_test = x_test.reshape((x_test.shape[0], 28 * 28)) / 255.0

    y_train = np.zeros((_y_train.shape[0], 10))
    y_train[np.arange(_y_train.shape[0]), _y_train] = 1

    y_test = np.zeros((_y_test.shape[0], 10))
    y_test[np.arange(_y_test.shape[0]), _y_test] = 1

    return (x_train, y_train), (x_test, y_test)


(x_train, y_train), (x_test, y_test) = load_mnist()

In [ ]:
def sigmoid(z: NDArray[float]):
    return 1.0 / (1.0 + np.exp(-z))


def sigmoid_prime(z: NDArray[float]):
    # Derivative of the sigmoid
    return sigmoid(z) * (1 - sigmoid(z))

## Warm-Up
Implement a numerically stable version of softmax.  
In general, softmax is defined as  
$$\text{softmax}(x_1, x_2, \ldots, x_n) = (\frac{e^{x_1}}{\sum_i{e^{x_i}}}, \frac{e^{x_2}}{\sum_i{e^{x_i}}}, \ldots, \frac{e^{x_n}}{\sum_i{e^{x_i}}})$$  
However, taking $e^{1000000}$ can result in NaN.  
Can you implement softmax so that the highest power to which e will be risen will be at most $0$ and the predictions will be mathematically equivalent?  

Hint: <sub><sub><sub>sǝnlɐʌ llɐ ɯoɹɟ ʇᴉ ʇɔɐɹʇqns  puɐ ᴉ‾x ʇsǝƃɹɐl ǝɥʇ ǝʞɐʇ</sub></sub></sub>

In [ ]:
def unstable_softmax(x: NDArray[float], axis: int = -1):
    e = np.exp(x)
    return e / np.sum(e, axis=axis, keepdims=True)


def stable_softmax(x: NDArray[float], axis: int = -1):
    ## TODO
    ###{
    e = np.exp(x -x.max())
    return e / np.sum(e, axis=axis, keepdims=True)
    # pass
    ###}


### TESTS ###
def test_one(x: NDArray[float], y: NDArray[float], proc_fn: Callable[[NDArray[float]], NDArray[float]]):
    r = stable_softmax(x)
    assert r.shape == x.shape
    r = proc_fn(r)
    assert r.shape == y.shape
    diff = np.mean(np.abs(r - y))
    assert diff <= 1e-5, r


x1 = np.random.rand(100, 32).astype(np.float64)
test_one(x1, np.ones(100), lambda x: x.sum(-1))
test_one(x1, unstable_softmax(x1), lambda x: x)


x2 = np.ones((100, 32), dtype=np.float64) * 1e6
test_one(x2, np.ones_like(x2) / x2.shape[-1], lambda x: x)
### TESTS END ###

In [ ]:
class Network(object):
    def __init__(self, sizes: List[int]):
        # initialize biases and weights with random normal distr.
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y) for y in sizes[1:]]
        self.weights = [np.random.randn(x, y) for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a: NDArray[float]) -> NDArray[float]:
        # Run the network on a single case
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(a @ w + b)

        return a

    def update_mini_batch(
        self, x_mini_batch: NDArray[float], y_mini_batch: NDArray[float], eta: float
    ) -> None:
        # Update network weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch.
        # eta is the learning rate

        nabla_b, nabla_w = self.backprop(x_mini_batch, y_mini_batch)

        self.weights = [w - eta * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - eta * nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(
        self, x: NDArray[float], y: NDArray[float]
    ) -> Tuple[List[NDArray[float]], List[NDArray[float]]]:
        # For a single input (x,y) return a tuple of lists.
        # First contains gradients over biases, second over weights.

        assert len(x.shape) == 2  # batch, features
        assert len(y.shape) == 2  # batch, classes
        assert x.shape[0] == y.shape[0]

        # First initialize the list of gradient arrays
        delta_nabla_b = []
        delta_nabla_w = []

        # Then go forward remembering each layer input and value
        # before sigmoid activation
        layer_input = []
        before_act = []
        for w, b in zip(self.weights, self.biases):
            layer_input.append(x)
            x = x @ w + b
            before_act.append(x)
            x = sigmoid(x)

        # Now go backward from the final cost applying backpropagation
        diff = self.cost_derivative(output_activations=x, y=y)
        for linp, bef_act, w, b in reversed(
            list(zip(layer_input, before_act, self.weights, self.biases))
        ):
            diff = sigmoid_prime(bef_act) * diff
            delta_nabla_w.append(linp.T @ diff)
            delta_nabla_b.append(np.sum(diff, axis=0))
            diff = diff @ w.T

        delta_nabla_w = reversed(delta_nabla_w)
        delta_nabla_b = reversed(delta_nabla_b)

        # Check shapes
        delta_nabla_b = list(delta_nabla_b)
        delta_nabla_w = list(delta_nabla_w)
        assert len(delta_nabla_b) == len(self.biases), (
            len(delta_nabla_b),
            len(self.biases),
        )
        assert len(delta_nabla_w) == len(self.weights), (
            len(delta_nabla_w),
            len(self.weights),
        )
        for lid in range(len(self.weights)):
            assert delta_nabla_b[lid].shape == self.biases[lid].shape, (
                delta_nabla_b[lid].shape,
                self.biases[lid].shape,
            )
            assert delta_nabla_w[lid].shape == self.weights[lid].shape, (
                delta_nabla_w[lid].shape,
                self.weights[lid].shape,
            )

        return delta_nabla_b, delta_nabla_w

    def evaluate(
        self, x_test_data: NDArray[float], y_test_data: NDArray[float]
    ) -> float:
        # Count the number of correct answers for test_data
        test_results = [
            (
                np.argmax(self.feedforward(x_test_data[i].reshape(1, 784)), axis=-1),
                np.argmax(y_test_data[i], axis=-1),
            )
            for i in range(len(x_test_data))
        ]
        # return accuracy
        return np.mean([int((x == y).item()) for (x, y) in test_results]).item()

    def cost_derivative(
        self, output_activations: NDArray[float], y: NDArray[float]
    ) -> NDArray[float]:
        assert output_activations.shape == y.shape, (output_activations.shape, y.shape)
        return (output_activations - y) / len(y)  # moved here from update_mini_batch

    def optimize(
        self,
        training_data: Tuple[NDArray[float], NDArray[float]],
        epochs: int,
        mini_batch_size: int,
        eta: float,
        test_data: Optional[Tuple[NDArray[float], NDArray[float]]] = None,
    ) -> None:
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        epoch_bar = tqdm(range(epochs), desc="Epoch")
        for j in epoch_bar:
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[
                    i * mini_batch_size : (i * mini_batch_size + mini_batch_size)
                ]
                y_mini_batch = y_train[
                    i * mini_batch_size : (i * mini_batch_size + mini_batch_size)
                ]
                self.update_mini_batch(x_mini_batch, y_mini_batch, eta)
            if test_data:
                epoch_bar.set_description_str(
                    "Epoch: {0}, Accuracy: {1}".format(j, self.evaluate(x_test, y_test))
                )
            else:
                epoch_bar.set_description_str("Epoch: {0}".format(j))

        return self.evaluate(x_test, y_test)


def evaluate_model(
    model_name: str,
    model_constructor: Callable[[List[int]], Network],
    result_storage: Dict[str, List[Any]],
    layers: List[int] = [784, 30, 10],
):
    # Remove logs from the previous evaluation of the same model
    result_storage["MODEL"] = [
        m for m in result_storage.get("MODEL", []) if m != model_name
    ]
    result_storage["LR"] = [
        lr
        for m, lr in zip(result_storage.get("MODEL", []), result_storage.get("LR", []))
        if m != model_name
    ]
    result_storage["ACC_STD"] = [
        acc_std
        for m, acc_std in zip(
            result_storage.get("MODEL", []), result_storage.get("ACC_STD", [])
        )
        if m != model_name
    ]
    result_storage["ACC"] = [
        acc
        for m, acc in zip(
            result_storage.get("MODEL", []), result_storage.get("ACC", [])
        )
        if m != model_name
    ]

    for lr in [0.001, 0.01, 0.1, 1.0, 10.0]:

        print(f"Checking with lr = {lr}")
        np.random.seed(42)
        accuracy_list = []
        for i in range(3):
            network = model_constructor(layers)
            accuracy = network.optimize(
                (x_train, y_train),
                epochs=10,
                mini_batch_size=100,
                eta=lr,
                test_data=(x_test, y_test),
            )
            accuracy_list.append(accuracy)

        result_storage["MODEL"].append(model_name)
        result_storage["LR"].append(lr)
        result_storage["ACC_STD"].append(
            f"{np.mean(accuracy_list) * 100:2.1f}% +- {np.std(accuracy_list) * 100:.1f}"
        )

        result_storage["ACC"].append(np.mean(accuracy_list))

    df = pd.DataFrame(result_storage).sort_values("ACC", ascending=False)
    df = df[[c for c in df.columns if c != "ACC"]]
    return df


RESULTS = {}
evaluate_model(
    model_name="Base",
    model_constructor=lambda x: Network(x),
    result_storage=RESULTS,
)

## Task 1
Use softmax instead of coordinate-wise sigmoid and use log-loss instead of MSE. Test to see if this improves convergence.   
Hint: When implementing backprop it might be easier to consider these two functions as a single block and not even compute the gradient over the softmax values.  
If you have problems, please see Appendix A.


In [ ]:
class Task1(Network):
    def __init__(self, sizes: List[int]):
        # initialize biases and weights with random normal distr.
        super().__init__(sizes=sizes)

    def feedforward(self, a: NDArray[float]) -> NDArray[float]:
        # Run the network on a single case

        ## TODO
        ###{
        pass
        ###}

        return a

    def backprop(
        self, x: NDArray[float], y: NDArray[float]
    ) -> Tuple[List[NDArray[float]], List[NDArray[float]]]:
        # For a single input (x,y) return a tuple of lists.
        # First contains gradients over biases, second over weights.

        assert len(x.shape) == 2  # batch, features
        assert len(y.shape) == 2  # batch, classes
        assert x.shape[0] == y.shape[0]

        ##TODO
        ###{
        pass
        ###}

        # Check shapes
        delta_nabla_b = list(delta_nabla_b)
        delta_nabla_w = list(delta_nabla_w)
        assert len(delta_nabla_b) == len(self.biases), (
            len(delta_nabla_b),
            len(self.biases),
        )
        assert len(delta_nabla_w) == len(self.weights), (
            len(delta_nabla_w),
            len(self.weights),
        )
        for lid in range(len(self.weights)):
            assert delta_nabla_b[lid].shape == self.biases[lid].shape, (
                delta_nabla_b[lid].shape,
                self.biases[lid].shape,
            )
            assert delta_nabla_w[lid].shape == self.weights[lid].shape, (
                delta_nabla_w[lid].shape,
                self.weights[lid].shape,
            )

        return delta_nabla_b, delta_nabla_w

    def cost_derivative(
        self, output_activations: NDArray[float], y: NDArray[float]
    ) -> NDArray[float]:
        assert output_activations.shape == y.shape, (output_activations.shape, y.shape)
        ## TODO
        ###{
        pass
        ###}


evaluate_model(
    model_name="SoftMax",
    model_constructor=lambda x: Task1(x),
    result_storage=RESULTS,
)

## Task 2
Implement L2 regularization and add momentum to the SGD algorithm. Play with different amounts of regularization and momentum. See if this improves accuracy/convergence.  
A few notes:
* do not regularize the biases
* you can see an example pseudocode here [pytorch.org/docs/stable/generated/torch.optim.SGD.html](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html)

In [ ]:
class Task2(Network):
    def __init__(
        self, sizes: List[int], l2_factor: float = 1e-4, momentum: float = 0.05
    ):
        # initialize biases and weights with random normal distr.
        super().__init__(sizes=sizes)
        self.l2_factor = l2_factor
        self.momentum = momentum
        ## TODO
        ####{
        pass
        ###}

    def update_mini_batch(
        self, x_mini_batch: NDArray[float], y_mini_batch: NDArray[float], eta: float
    ) -> None:
        # Update network weights and biases by applying a single step
        # of gradient descent (with momentum and l2 regularization) using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch.
        # eta is the learning rate
        ## TODO
        ###{
        pass
        ###}


evaluate_model(
    model_name="L2&Momentum",
    model_constructor=lambda x: Task2(x),
    result_storage=RESULTS,
)

## Task 3 (optional)
Implement Adagrad or AdamW (currently popular in LLM training), dropout, and some simple data augmentations (e.g. tiny rotations/shifts etc.). Again, test to see how these changes improve accuracy/convergence.  
In case you want to learn about AdamW you can check the official [PyTorch docummentation](https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html) that contains pseudocode and the original paper [Decoupled Weight Decay Regularization](https://arxiv.org/abs/1711.05101).  
In Appendix B you can find a simplified version of AdaGrad.  
Below you can find brief information regarding the dropout:  

During the training phase, we want to make some activations $0$.
It is usually implemented by zeroing each activation in the considered layer with probability $p$, and multiplying other activations by $\frac{1}{1-p}$.

In [ ]:
class Task3Optimizer(Network):
    def __init__(
        self,
        sizes: List[int],
        weight_decay: float = 0.01,
        beta_1: float = 0.9,
        beta_2: float = 0.95,
        eps=1e-5,
    ):
        # initialize biases and weights with random normal distr.
        super().__init__(sizes=sizes)
        self.weight_decay = weight_decay
        self.beta_1 = beta_1
        self.beta_2 = beta_2
        self.eps = eps
        ## TODO
        ####{
        pass
        ###}

    def update_mini_batch(
        self, x_mini_batch: NDArray[float], y_mini_batch: NDArray[float], eta: float
    ) -> None:
        # Update network weights and biases by applying a single step
        # of gradient descent (with momentum and weight decay) using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch.
        # eta is the learning rate
        ## TODO
        ###{
        pass
        ###}


evaluate_model(
    model_name="Optimizer",
    model_constructor=lambda x: Task3Optimizer(x),
    result_storage=RESULTS,
)

In [ ]:
# Place for remaining parts of task 3

## Task 4
Try adding extra layers to the network. Again, test how the changes you introduced affect accuracy/convergence. As a start, you can try this architecture: [784,100,30,10]  


In [ ]:
## TODO

# Appendix

## Appendix A - Log-loss and softmax

Let's compute the following derivative
$$\frac{\partial\left(\sum_{i}{y_{b, i}\log{(\text{prediction}_{b, i}})}\right)}{\partial z_{b, f}}$$


where
$$
y_{b, i} = \begin{cases}
1 & \text{if $b$'th image belongs to $i$'th class}\\
0 & \text{otherwise}
\end{cases}
$$


To do so we can use the chain rule:
$$
\frac{\partial f(g(x))}{\partial x} = \frac{\partial f(g(x))}{\partial g(x)}  \frac{\partial g(x)}{\partial x}
$$
the rule for the sum  
$$
\frac{\partial (f(x) + g(x))}{\partial x} = \frac{\partial f(x)}{\partial x} +   \frac{\partial g(x)}{\partial x}
$$
and the fact that the derivative of natural logarithm is
$$
\frac{\partial \log(x)}{\partial x} = \frac{1}{x}
$$

Using those rules we can observe that indeed:

$$
\frac{\partial  \left(\sum_{i}{y_{b, i}\log{(\text{prediction}_{b, i}})}\right)}{\partial z_{b, f}} = \sum_{i}{y_{b,i} \frac{1}{\text{prediction}_{b, i}} \frac{ \partial \text{prediction}_{b, i}}{\partial z_{b, f}}}
$$



Let us review the multiplication rule:
$$
\frac{\partial (f(x)g(x))}{\partial x} = \frac{\partial f(x)}{\partial x}g(x)  + f(x)\frac{\partial g(x)}{\partial x}
$$
and that
$$
\frac{\partial e^x}{\partial x} = e^x
$$

From the definition:
$$
\text{prediction}_{b, f} = \frac{e^{z_{b, f}}}{\sum_{i}{e^{z_{b, i}}}}
$$


We can observe that for $k \not= f$
$$
\frac{ \partial \text{prediction}_{b, k}}{\partial z_{b, f}}
= 0 - \frac{e^{z_{k, b}} e^{z_{b, f}} }{\left(\sum_{i}{e^{z_{b, i}}}\right)^2}
$$
and that
$$
\frac{ \partial \text{prediction}_{b, f}}{\partial z_{b, f}}
= \frac{e^{z_{b, f}}}{\sum_{i}{e^{z_{b, i}}}} - \left(\frac{e^{z_{b, f}}}{\sum_{i}{e^{z_{b, i}}}}\right)^2
$$

Therefore
$$
y_{b,i} \frac{1}{\text{prediction}_{b, i}} \frac{ \partial \text{prediction}_{b, i}}{\partial z_{b, f}} = \begin{cases}
y_{b,i}\left(-\frac{e^{z_{b, f}}}{\sum_{i}{e^{z_{b, i}}}}\right) & \text{if } i \not= f\\
y_{b,i}\left(1 - \frac{e^{z_{b, f}}}{\sum_{i}{e^{z_{b, i}}}}\right) & \text{if } i = f\\
\end{cases}
$$
in other words
$$
y_{b,i} \frac{1}{\text{prediction}_{b, i}} \frac{ \partial \text{prediction}_{b, i}}{\partial z_{b, f}} = \begin{cases}
y_{b,i}\left(-\text{prediction}_{b, f}\right) & \text{if } i \not= f\\
y_{b,i}\left(1 - \text{prediction}_{b, f}\right) & \text{if } i = f\\
\end{cases}
$$

As we just sum over $i$, and for each $b$ there is exactly one $i$ such that $y_{b, i} = 1$ and for $j \not= i$ $y_{b, j} = 0$, therefore  in the end we have that
$$
\frac{\partial  \left(\sum_{i}{y_{b, i}\log{(\text{prediction}_{b, i}})}\right)}{\partial z_{b, f}} = y_{b, f} - \text{prediction}_{b, f}
$$
what can be written as:
$$
y - prediction
$$

In the code, we are going to use
$$
prediction - y
$$
as our loss is
$$-\sum_{b}\sum_{i}{y_{b, i}\log{(\text{prediction}_{b, i}})}$$

## Appendix B - Adagrad (simplified version)

Let $p_1, \ldots, p_n$ be all parameters in our model (weights and biases).  
For parameter $p_i$ we maintain an array $G_i$ (can be set to $0$ initially).
Let $\text{LOSS}$ be our loss without L2.   
We update $G_i$ and $p_i$ each training step as follows:  
$$
G_i = G_i +  \left(\frac{\partial \text{LOSS}}{\partial p_i}\right)^2\\
p_i = p_i - \frac{\eta}{\sqrt{\left(G_i + \epsilon\right)}}\frac{\partial \text{LOSS}}{\partial p_i}
$$